In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pandas_profiling
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
df_london = pd.read_csv("London Airbnb Listing Detailed.csv")
df_london.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,11551.0,https://www.airbnb.com/rooms/11551,2.019091e+13,2019-09-15,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,...,t,f,strict_14_with_grace_period,f,t,2.0,2.0,0.0,0.0,1.59
1,13913.0,https://www.airbnb.com/rooms/13913,2.019091e+13,2019-09-15,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,...,f,f,moderate,f,f,3.0,1.0,2.0,0.0,0.14
2,15400.0,https://www.airbnb.com/rooms/15400,2.019091e+13,2019-09-15,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Bright Chelsea Apartment This is a bright one...,Lots of windows and light. St Luke's Gardens ...,romantic,It is Chelsea.,...,t,f,strict_14_with_grace_period,t,t,1.0,1.0,0.0,0.0,0.73


In [3]:
df_london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85274 entries, 0 to 85273
Columns: 106 entries, id to reviews_per_month
dtypes: float64(35), object(71)
memory usage: 69.0+ MB


In [4]:
df_london.select_dtypes("object").columns

Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'minimum

In [5]:
df_london.select_dtypes("object").columns

Index(['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'minimum

In [6]:
floatcol = ['id', 'scrape_id', 'medium_url', 'xl_picture_url',
       'host_listings_count', 'latitude', 'longitude', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',
       'minimum_nights', 'maximum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

objectcol = ['listing_url', 'last_scraped', 'name', 'summary', 'space',
       'description', 'experiences_offered', 'neighborhood_overview', 'notes',
       'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url',
       'picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'minimum_minimum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'calendar_last_scraped', 'first_review',
       'last_review', 'requires_license', 'license', 'jurisdiction_names',
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']

X = ['host_listings_count', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'guests_included',
       'minimum_nights', 'maximum_nights', 'availability_365', 'number_of_reviews',
       'number_of_reviews_ltm', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count',
       'reviews_per_month','experiences_offered', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed','is_location_exact',
       'property_type', 'room_type', 'bed_type', 'amenities', 'security_deposit', 'cleaning_fee',
       'extra_people',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
       'last_review', 'requires_license',
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']

excludefornow = set(floatcol + objectcol).difference(set(X))

excludefornow = list(excludefornow)
excludefornow.sort()
excludefornow

['access',
 'availability_30',
 'availability_60',
 'availability_90',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms',
 'calendar_last_scraped',
 'calendar_updated',
 'city',
 'country',
 'country_code',
 'description',
 'first_review',
 'has_availability',
 'host_about',
 'host_id',
 'host_location',
 'host_name',
 'host_neighbourhood',
 'host_picture_url',
 'host_since',
 'host_thumbnail_url',
 'host_url',
 'house_rules',
 'id',
 'interaction',
 'jurisdiction_names',
 'last_scraped',
 'latitude',
 'license',
 'listing_url',
 'longitude',
 'market',
 'maximum_maximum_nights',
 'maximum_minimum_nights',
 'medium_url',
 'minimum_maximum_nights',
 'minimum_minimum_nights',
 'monthly_price',
 'name',
 'neighborhood_overview',
 'neighbourhood',
 'neighbourhood_group_cleansed',
 'notes',
 'picture_url',
 'price',
 'scrape_id',
 'smart_location',
 'space',
 'square_feet',
 'state',
 'street',
 's

In [ ]:
columns_to_delete = ["calculated_host_listings_count_entire_homes", "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms", 
                     "host_acceptance_rate", "host_listings_count", "last_scraped", "maximum_maximum_nights", "medium_url", "minimum_maximum_nights", "host_neighbourhood",
                     "minimum_minimum_nights", "neighbourhood_group_cleansed", "scrape_id", "square_feet", "thumbnail_url", "xl_picture_url", "listing_url", "name",
                     "summary", "space", "description", "neighborhood_overview", "notes", "transit", "access", "interaction", "house_rules", "picture_url", 
                     "host_id", "host_url", "host_name", "host_location", "host_about", "host_response_time", "host_thumbnail_url", "host_picture_url", "street", "state",
                     "zipcode", "market", "smart_location", "country_code", "country", "calendar_updated", "has_availability", "calendar_last_scraped", "license", 
                     "jurisdiction_names", "maximum_minimum_nights", "availability_60", "availability_90", "availability_30"]

In [ ]:
diff = set(columns_to_delete).difference(set(excludefornow))
diff